In [30]:
from bs4 import BeautifulSoup
import requests
import datetime
import re
from pprint import pprint
import csv

In [19]:
headers = ['loan term', 'repayment schedule', 'disbursment date', 
           'exchange loss', 'field partner', 'interest', 'risk rating']
kiva_info = []

In [3]:
number = 1507000
while (number <=1507005):
    baseurl = 'https://www.kiva.org/lend/'
    url = baseurl + str(number)
    print (url)
    number+=1
#response = requests.get(url, headers={'User-Agent': 'Chrome/65.0'})

https://www.kiva.org/lend/1507000
https://www.kiva.org/lend/1507001
https://www.kiva.org/lend/1507002
https://www.kiva.org/lend/1507003
https://www.kiva.org/lend/1507004
https://www.kiva.org/lend/1507005


In [10]:
url = 'https://www.kiva.org/lend/1508098'
response = requests.get(url, headers={'User-Agent': 'Chrome/65.0'})

In [ ]:
response.status_code

In [ ]:
print(response.text)

In [22]:
page = response.text

In [23]:
soup = BeautifulSoup(page, "lxml")

In [ ]:
print(soup)

In [ ]:
print (soup.prettify())

In [ ]:
soup.div.prettify()

In [ ]:
for div in soup.find_all('div'):
    print(div.prettify())

In [ ]:
#ac-title ac-container
#ac-loan-details-body-right.ac-body


In [ ]:
for element in soup.find_all(id='ac-loan-details-body-right'):
   print(element, '\n')

In [24]:
def funded_percent(soup):
    funded = (soup.find(class_='green-bolded inline')).text.strip()[0]
    if (funded == 'F'):
        return 100
    else:
        return int(funded)


In [ ]:
#funded = funded_percent(soup)
#funded

In [25]:
funded = funded_percent(soup)

loan = (soup.find(class_='loan-total').text.strip())

#This gets the str for the length of the loan term.
term = (soup.find(id='ac-loan-details-body-right').find(class_='green-bolded').text.strip())

#Gets the loan repayment schedule
schedule = (soup.find(id='ac-loan-details-body-right').find(id='repayment-schedule').text.strip().split(' ')[-1])

#Returns the date the loan was disbersed
disbursed = (soup.find(id='ac-loan-details-body-right').find(id='disbursed-date').text.strip().split(' ')[-3:])
disbursed = datetime.datetime.strptime(" ".join(disbursed),"%B %d, %Y").date()

#Returns currency exchange loss possibility
exchange_loss = (soup.find(id='ac-loan-details-body-right').find(id='currency-exchange-loss').text.strip().split(' ')[-1])

#Returns name of field partner
field_partner = (soup.find(id='ac-loan-details-body-right').find(id='field-partner').text.strip().split(' '))
partner = ' '.join(field_partner[0:4]).split(':')[0]
field_partner = ' '.join(field_partner[-2:])

#Returns if loan recipient is paying interest
interest = (soup.find(id='ac-loan-details-body-right').find(id='borrower-paying-interest').text.strip().split(' ')[-1])

#Calculates the numeric number of stars representing the risk rating
risk_stars = 0
for tag in (soup.find(id='ac-loan-details-body-right').find_all(class_='icon icon-star')):
    risk_stars+=1
for tag in (soup.find(id='ac-loan-details-body-right').find_all(class_='icon icon-half-star')):
    risk_stars +=.5


In [26]:
kiva_dict = dict(zip(headers, [term,
                               schedule,
                               disbursed,
                               exchange_loss,
                               [partner,field_partner],
                               interest,
                               risk_stars]))

In [27]:
kiva_info.append(kiva_dict)

In [28]:
pprint(kiva_info)

[{'disbursment date': datetime.date(2018, 3, 31),
  'exchange loss': 'Possible',
  'field partner': ['Facilitated by Field Partner', 'Juhudi Kilimo'],
  'interest': 'Yes',
  'loan term': '15 months',
  'repayment schedule': 'Monthly',
  'risk rating': 3.5},
 {'disbursment date': datetime.date(2018, 3, 22),
  'exchange loss': 'Possible',
  'field partner': ['Facilitated by Field Partner', 'Juhudi Kilimo'],
  'interest': 'Yes',
  'loan term': '14 months',
  'repayment schedule': 'Monthly',
  'risk rating': 3.5}]


In [29]:
kiva_info.to_csv('testing_kiva.csv')

AttributeError: 'list' object has no attribute 'to_csv'

In [32]:
#This does write to a CSV file. I am not 100% sure it writes exactly as I want it to.
with open('kiva_1.csv', 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(kiva_info)

In [48]:
#close but not quite doing what I would want it to
with open('kiva_1.csv', 'r') as myfile:
    read = csv.reader(myfile, quoting=csv.QUOTE_ALL)
    kiva_2 = {rows[0]:rows[1] for rows in read}

dict_keys(["{'loan term': '15 months', 'repayment schedule': 'Monthly', 'disbursment date': datetime.date(2018, 3, 31), 'exchange loss': 'Possible', 'field partner': ['Facilitated by Field Partner', 'Juhudi Kilimo'], 'interest': 'Yes', 'risk rating': 3.5}"])